In [ ]:
import Models
import Common
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import datetime
from sklearn.model_selection import train_test_split
from DatasetLoader import DatasetLoader

%load_ext tensorboard

In [ ]:
# experimental, see
# https://silpara.medium.com/stop-tensorflow-from-consuming-all-the-gpu-memory-8bfda3de970d

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print("Type: {}, name: {}".format(gpu.name, gpu.device_type))
    tf.config.experimental.set_memory_growth(gpu, True)

## Load Model

### Fresh Model

In [ ]:
model = Models.create_2DCNN_MLP()

In [ ]:
model = Models.create_3DCNN()

In [ ]:
model = Models.create_LSTM()

In [ ]:
model = Models.create_Transfer_LSTM((40, 128, 128, 3))

In [ ]:
model = Models.lstm_test((40, 128, 128, 3))

### Load existing model

In [ ]:
model = tf.keras.models.load_model("models/2DCNN_MLP_grayscale")

## Training

In [ ]:
train = Common.evaluate_dataset("dataset/train/", shuffle=True)
X_train = train.path
y_train = tf.keras.utils.to_categorical(train.category, 51)

In [ ]:
test = Common.evaluate_dataset("dataset\\test\\")
X_test = test.path
y_test = tf.keras.utils.to_categorical(test.category, 51)

In [ ]:
dsl = DatasetLoader(X_train, y_train, batch_size=16, proprocessing="InceptionResNetV2")
verify_dsl = DatasetLoader(X_test, y_test, batch_size=16, proprocessing="InceptionResNetV2")

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit(
    dsl, 
    validation_data=verify_dsl,
    epochs=100,
    validation_freq=10,
    use_multiprocessing=True, 
    workers=4, 
    callbacks=[tensorboard_callback])

In [ ]:
model.save("models/" + model.name)

In [ ]:
%tensorboard --logdir logs/fit --port 6006 --host localhost